In [8]:
# ✅ Célula completa para conexão com MinIO e PostgreSQL (com nome correto do container)

# 🔗 Importação de bibliotecas necessárias
from minio import Minio
from minio.error import S3Error
import psycopg2
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

# 🔗 Configuração da conexão com o MinIO
MINIO_ENDPOINT = "localhost:9000"      # 🔧 Endereço do MinIO (ajuste se necessário)
MINIO_ACCESS_KEY = "minioadmin"        # 🔧 Acesso MinIO
MINIO_SECRET_KEY = "minioadmin"        # 🔧 Senha MinIO

# 🔗 Configuração da conexão com o PostgreSQL
# ⚠️ O hostname correto é o nome do container: postgres_db
PG_HOST = "postgres_db"
PG_PORT = "5432"
PG_DATABASE = "postgres"
PG_USER = "postgres"
PG_PASSWORD = "senhasegura"

# 🔗 Conexão com o MinIO
try:
    client = Minio(
        MINIO_ENDPOINT,
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=False
    )
    print("🟢 Conectado ao MinIO com sucesso.")
except Exception as e:
    print(f"❌ Erro na conexão com o MinIO: {e}")

# 🔗 Conexão com o PostgreSQL
try:
    conn = psycopg2.connect(
        host=PG_HOST,
        port=PG_PORT,
        dbname=PG_DATABASE,
        user=PG_USER,
        password=PG_PASSWORD
    )
    print("🟢 Conectado ao PostgreSQL com sucesso.")
except Exception as e:
    print(f"❌ Erro na conexão com o PostgreSQL: {e}")


🟢 Conectado ao MinIO com sucesso.
🟢 Conectado ao PostgreSQL com sucesso.


In [9]:
# ✅ Célula completa — Seleção ou Criação de Projeto + Pipeline com Contadores Dinâmicos

# 🔗 Importações necessárias
from minio import Minio
from minio.error import S3Error
from minio.commonconfig import CopySource
import psycopg2
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# 🔗 Conexão com MinIO
MINIO_ENDPOINT = "minio:9000"
MINIO_ACCESS_KEY = "admin"          # 🔧 Ajustar conforme ambiente
MINIO_SECRET_KEY = "senhasegura"    # 🔧 Ajustar conforme ambiente

client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)

# 🔗 Definir buckets
bucket_recepcao = "recepcao-raw"
bucket_destino = "storage-unique"

# 🗂️ Verificar e criar bucket de destino se não existir
if not client.bucket_exists(bucket_destino):
    client.make_bucket(bucket_destino)
    print(f"✅ Bucket '{bucket_destino}' criado.")
else:
    print(f"ℹ️ Bucket '{bucket_destino}' já existe.")

# 🔍 Função para buscar projetos ativos no PostgreSQL
def fetch_projects():
    query = """
        SELECT prefix, project_name
        FROM projects_registry
        WHERE active = TRUE
        ORDER BY project_name;
    """
    df = pd.read_sql(query, conn)
    if df.empty:
        return []
    return [f"{row.prefix} - {row.project_name}" for _, row in df.iterrows()]

# 🔧 Widgets para seleção de projeto
dropdown = widgets.Dropdown(
    options=fetch_projects(),
    description='Projeto:',
    layout=widgets.Layout(width='50%')
)

# 🔧 Widgets para criação de novo projeto
input_name = widgets.Text(description="Nome:")
input_prefix = widgets.Text(description="Prefixo:")
input_description = widgets.Text(description="Descrição:")

button_create = widgets.Button(
    description="Criar Projeto",
    button_style='info'
)

# 🔧 Botão para executar movimentação
button_run = widgets.Button(
    description="Iniciar Movimentação",
    button_style='success'
)

# 🎯 Callback para criar projeto
def on_create_project(b):
    prefix = input_prefix.value.strip().upper()
    name = input_name.value.strip()
    desc = input_description.value.strip()

    if not prefix or not name:
        print("❌ Prefixo e Nome do Projeto são obrigatórios.")
        return

    cursor = conn.cursor()
    cursor.execute("SELECT 1 FROM projects_registry WHERE prefix = %s;", (prefix,))
    exists = cursor.fetchone()

    if exists:
        print(f"❌ Prefixo '{prefix}' já existe. Escolha outro.")
    else:
        cursor.execute("""
            INSERT INTO projects_registry (prefix, project_name, description, active, created_at)
            VALUES (%s, %s, %s, TRUE, CURRENT_TIMESTAMP);
        """, (prefix, name, desc))
        conn.commit()
        print(f"✅ Projeto '{name}' com prefixo '{prefix}' criado com sucesso.")

        # 🔄 Atualizar dropdown
        dropdown.options = fetch_projects()

    cursor.close()

# 🎯 Callback para movimentação
def on_run_clicked(b):
    if dropdown.value:
        prefix = dropdown.value.split(" - ")[0]
        project_name = dropdown.value.split(" - ")[1]
        print(f"🗂️ Projeto selecionado: {project_name} (Prefixo: {prefix})")
    else:
        print("❌ Nenhum projeto selecionado.")
        return

    # 📦 Listar arquivos no bucket de recepção
    print(f"🔍 Listando arquivos no bucket '{bucket_recepcao}'...")
    objects = list(client.list_objects(bucket_recepcao, recursive=True))
    print(f"➡️ {len(objects)} arquivos encontrados no bucket '{bucket_recepcao}'.")

    # 🚩 Contadores
    copiados = 0
    repetidos = 0

    # 🚚 Loop de movimentação com verificação dupla e barra de progresso com contadores
    with tqdm(total=len(objects), desc="Copiando arquivos", unit="arquivo") as pbar:
        for obj in objects:
            source_path = obj.object_name

            # 🔁 Gerar novo caminho no bucket de destino
            if source_path.startswith("recepcao-raw/FIAP_PI/"):
                relative_path = source_path.replace("recepcao-raw/FIAP_PI/", "")
            else:
                relative_path = source_path

            new_path = f"{prefix}/{relative_path}"

            # 🔍 Verificar na storage_audit (banco)
            cursor = conn.cursor()
            cursor.execute("""
                SELECT 1 FROM storage_audit WHERE full_path = %s;
            """, (new_path,))
            exists_in_audit = cursor.fetchone() is not None
            cursor.close()

            # 🔍 Verificar no bucket destino
            try:
                client.stat_object(bucket_destino, new_path)
                exists_in_bucket = True
            except S3Error as err:
                if err.code == "NoSuchKey":
                    exists_in_bucket = False
                else:
                    raise

            # 🔎 Decidir se copia
            if exists_in_audit and exists_in_bucket:
                repetidos += 1
                tqdm.write(f"⏭️ Arquivo já presente no audit e no bucket: {new_path}")
            else:
                try:
                    # 📤 Copiar usando CopySource
                    client.copy_object(
                        bucket_destino,
                        new_path,
                        CopySource(bucket_recepcao, source_path)
                    )
                    copiados += 1
                    tqdm.write(f"✅ Arquivo copiado: {source_path} → {new_path}")

                    # 🗃️ Registrar na storage_audit (se não estiver)
                    if not exists_in_audit:
                        cursor = conn.cursor()
                        cursor.execute("""
                            INSERT INTO storage_audit (
                                prefix, project_name, bucket, full_path, filename,
                                size_bytes, upload_date, source_bucket
                            )
                            VALUES (%s, %s, %s, %s, %s, %s, CURRENT_TIMESTAMP, %s);
                        """, (
                            prefix,
                            project_name,
                            bucket_destino,
                            new_path,
                            new_path.split("/")[-1],
                            obj.size,
                            bucket_recepcao
                        ))
                        conn.commit()
                        cursor.close()

                except Exception as e:
                    tqdm.write(f"❌ Erro ao copiar {source_path}: {e}")

            # 🔄 Atualiza a barra com os contadores em tempo real
            pbar.set_postfix(copiados=copiados, repetidos=repetidos)
            pbar.update(1)

    # 🔔 Resumo final
    print(f"✅ Finalizado: {copiados} arquivos copiados, {repetidos} arquivos já existentes.")
    
# 🔗 Conectar callbacks
button_create.on_click(on_create_project)
button_run.on_click(on_run_clicked)

# 🗂️ Montagem da interface
interface = widgets.VBox([
    widgets.HTML("<b>Selecione um Projeto Existente:</b>"),
    dropdown,
    widgets.HTML("<b>Ou Cadastre um Novo Projeto:</b>"),
    input_name,
    input_prefix,
    input_description,
    button_create,
    widgets.HTML("<b>Após selecionar o projeto, execute a movimentação:</b>"),
    button_run
])

# 🚀 Exibir a interface
display(interface)


ℹ️ Bucket 'storage-unique' já existe.


/tmp/ipykernel_252712/3355056888.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Copiando arquivos:   0%|          | 0/23883 [00:00<?, ?arquivo/s]